In [62]:
%config Completer.use_jedi = False

it is not so convenient to retrieve history data of non-us listed stocks. 
the same stock might be listed in several vennues with the same symbol. to query the data in yfinance, it therefore requires to specify the location. (if londong, applend '.L', amsterdam '.AM', paris '.PA'). i still need to comeup with a way to automate it.  

ISIN is unique to the stock, there is probably a way to query based on ISIN information. 

In [63]:
import yfinance as yf

In [64]:
sys.path.append("../scr")

In [65]:
import sys
from datetime import datetime, timedelta

import altair as alt
import numpy as np
import pandas as pd
import panel as pn
import yfinance as yf
from altair import datum
from utility import (
    plot_transactions,
    plot_transactions_2,
    read_ticker_ts,
    read_transactions,
)

from datetime import date, timedelta

In [66]:
data = yf.download(['AF.PA', 'RDSA.AS', 'IITU.L', 'LTMAQ', "VUSA.L",
                    'INRG.LN'], start='2021-01-01', end='2021-03-01')['Close']

[*********************100%***********************]  6 of 6 completed

1 Failed download:
- INRG.LN: No data found, symbol may be delisted


In [67]:
yf.download(['INRG.L','INRG.LN'])['Close']

[*********************100%***********************]  2 of 2 completed

1 Failed download:
- INRG.LN: No data found, symbol may be delisted


,INRG.L,INRG.LN
Date,,
2007-07-11,13.28050,NaN
2007-07-12,13.46480,NaN
2007-07-13,13.54660,NaN
2007-07-16,13.61300,NaN
2007-07-17,13.41430,NaN
...,...,...
2018-09-10,5.24125,NaN
2018-09-11,5.24125,NaN
2018-09-12,5.24125,NaN


# convert dataframe to dictionary

In [155]:
fln = "../data/trasaction_history_18022021.csv"
tr = read_transactions(fln)

tr_sub = tr[['Ticker', 'Currency (Price / share)', 'ISIN']].drop_duplicates()
tr_sub.rename({'Currency (Price / share)': 'Currency'}, axis=1, inplace=True)
tr_sub = tr_sub.dropna(subset=['Ticker'])
# tickers = tr_sub.set_index('Ticker').T.to_dict()

In [172]:
us_stocks = tr_sub.loc[tr_sub['Currency']=='USD', 'Ticker'].to_list()

In [152]:
tr_sub.Currency.unique()

array(['USD', 'GBX', 'EUR', 'GBP'], dtype=object)

In [153]:
start = tr.Time.min()
end = tr.Time.max() +  timedelta(1)

# get forex

In [131]:
from pandas_datareader import data

# data.DataReader(['DEXUSEU','DEXUKEU'], 'fred')

In [143]:
import pandas as pd
import pandas_datareader as dr

# retrieve market data of current ticker symbol
df_forex = dr.data.DataReader(
    ['GBPEUR%3DX', 'USDEUR%3DX', "AAPL"], data_source='yahoo', start=start, end=end)['Close']

In [173]:
tickers = tr.Ticker.dropna().unique()
tickers = tickers.tolist()

start = tr.Time.min()
end = tr.Time.max() +  timedelta(1)

data = yf.download(us_stocks, start, end)['Close']

[*********************100%***********************]  36 of 36 completed

1 Failed download:
- LTM: No data found, symbol may be delisted


In [159]:
yf.download(['CCL.L'], start, end)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-30,1190.000000,1249.154053,1070.0,1095.500000,1095.500000,5191852
2020-05-01,1066.000000,1074.000000,1000.0,1003.000000,1003.000000,1508790
2020-05-04,986.599976,1001.895020,893.0,964.400024,964.400024,3609555
2020-05-05,1006.000000,1035.600952,925.0,960.599976,960.599976,4279369
2020-05-06,942.200012,955.000000,903.0,908.799988,908.799988,2391386
...,...,...,...,...,...,...
2021-02-12,1249.000000,1274.500000,1191.5,1263.000000,1263.000000,1250749
2021-02-15,1280.000000,1363.000000,1269.0,1363.000000,1363.000000,936556
2021-02-16,1357.500000,1387.000000,1309.0,1367.000000,1367.000000,1546230


In [166]:
dr.data.DataReader( "AF.PA", start=start, end=end, data_source='yahoo')

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-05-04,4.540,4.280,4.540,4.299,3596266,4.299
2020-05-05,4.447,4.315,4.418,4.376,2173941,4.376
2020-05-06,4.383,4.203,4.354,4.203,2701066,4.203
2020-05-07,4.217,4.000,4.200,4.097,4138979,4.097
2020-05-08,4.290,4.154,4.200,4.213,2029244,4.213
...,...,...,...,...,...,...
2021-02-15,4.991,4.831,4.882,4.927,3796310,4.927
2021-02-16,4.999,4.810,4.840,4.918,3876369,4.918
2021-02-17,4.940,4.768,4.937,4.796,4545049,4.796
